In [95]:
import pandas as pd
import numpy as np
import sys
import re
import requests 
from bs4 import  BeautifulSoup
import json
from tqdm import tnrange, tqdm_notebook
from datetime import datetime
from tqdm import trange,tnrange, tqdm_notebook
import time
import polars as pl

In [96]:
import pyodbc
from requests_html import AsyncHTMLSession
asession = AsyncHTMLSession()
from datetime import date
from dateutil.relativedelta import relativedelta

In [97]:
idx = pd.IndexSlice
pd.options.display.float_format = '{:,.3f}'.format

# CRAWL BÁO CÁO TỪ VCI

In [84]:
url = 'https://www.vietcap.com.vn/api/cms-service/v1/page/analysis?is-all=true&page=0&size=200&direction=DESC&sortBy=date&language=2'
r = requests.get(url)
data = r.json()['data']['pagingGeneralResponses']['content']
df = pd.DataFrame(data)


In [89]:
filter_link = (~pd.isnull(df['file']))&(df['pageLink']!='market-commentary')
# &(~df['fname'].str.contains('Daily'))&(~df['fname'].str.lower().contains('monthly'))
link_pdf = df[filter_link]
link_pdf['fname'] = link_pdf['file'].apply(lambda x: x.split('/')[-1] if x else None)
filter_link = (~link_pdf['fname'].str.contains('Daily'))&(~link_pdf['fname'].str.contains('monthly'))
link_pdf = link_pdf[filter_link]

C:\Users\HP PAVILION\AppData\Local\Temp\ipykernel_2172\114233046.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link_pdf['fname'] = link_pdf['file'].apply(lambda x: x.split('/')[-1] if x else None)


In [93]:
for i in range(len(link_pdf)):
    #check if the file is already downloaded or not
    if link_pdf['fname'].iloc[i] not in files:
        try:
            url = link_pdf['file'].iloc[i]
            r = requests.get(url)
            with open(link_pdf['fname'].iloc[i], 'wb') as f:
                f.write(r.content)
            print(f'{link_pdf["fname"].iloc[i]} is downloaded')
        except:
            print(f'{link_pdf["fname"].iloc[i]} error')
    else:
        print(f'{link_pdf["fname"].iloc[i]} exist')

BeerSector-20240722.pdf is already downloaded
MacroNote-20240716.pdf is downloaded
RESectorFlash-20240715.pdf is downloaded
Vietcap_Midyear_Strategy_2024.pdf is already downloaded
FixedIncome-20240710-June2024.pdf is downloaded
PLX-20240710-BUY.pdf is downloaded
PTB-20240709-OP.pdf is already downloaded
SteelThoughtpiece-20240709.pdf is downloaded
MacroUpdate-20240705.pdf is downloaded
Monthly-20240702-June2024.pdf is downloaded
HDG-20240701-MP.pdf is already downloaded
STK-20240628-BUY.pdf is already downloaded
ACG-20240627-MP.pdf is downloaded
WaterSector-20240627.pdf is downloaded
POW-20240625-MP.pdf is already downloaded
MacroNote-20240624.pdf is downloaded
AGG-20240624-VisitNote.pdf is already downloaded
VEA-20240620-MP.pdf is already downloaded
BMP-20240618-OP.pdf is already downloaded
PNJ-20240612-Thoughtpiece.pdf is already downloaded
FixedIncome-20240612-May2024.pdf is downloaded
PDR-20240610-NR.pdf is already downloaded
SSI-20240610-OP.pdf is downloaded
GEX-20240607-OP.pdf is

# CRAWL BÁO CÁO MBKE

In [ ]:
import os
dir_path = 'D://MBKE'
file_list = os.listdir(dir_path)
list_file = pd.DataFrame(file_list, columns=['filename'])
list_file = list(list_file['filename'])

link = 'https://www.kelive.com/KimEng/servlet/PDFDownloadViaEmail?source=0&rid='
list_link = pd.DataFrame()
start = 56500
end = start + 500
list_link['id_file'] = range(start,end)
list_link['link'] = link + list_link['id_file'].astype(str)
r = requests.get(list_link.loc[0,'link'],timeout=2)
for i in tnrange(len(list_link)):
    try:
        r = requests.get(list_link.loc[i,'link'],timeout=2)
        try:
            with r:
                fname = ''
                if "Content-Disposition" in r.headers.keys():
                    fname = re.findall("filename=(.+)", r.headers["Content-Disposition"])[0]
                else:
                    fname = list_link.loc[i,'link'].split("/")[-1]
            list_link.loc[i,'filename'] = fname[1:-1]
            if list_link.loc[i,'filename'] not in list_file:
                with open("D://MBKE/"+list_link.loc[i,'filename'], "wb") as f:
                    f.write(r.content)
            else: print('có rồi')

        except requests.exceptions.RequestException as e:
            print(e)
    except requests.exceptions.RequestException as e:
        list_link.loc[i,'status'] = 'ERROR'
        print("Error", list_link.loc[i,'link'])


# XỬ LÝ DF CÔNG TY LISTED

In [6]:
df = pd.read_pickle('../DataSQL/df_merge')

In [7]:
nganh = nganh.rename(columns={'ticker':'Ticker'})

In [20]:
df.loc[idx['VIC',2023:,[9]],['st_debt','lt_debt','te','cce','net_st_inv']]/10**9

,,,st_debt,lt_debt,te,cce,net_st_inv
Ticker,year,quarter,,,,,
VIC,2023,9.000,"95,183.876","98,292.858","146,942.516","16,882.015","7,369.738"


In [9]:
df = pd.merge(df.reset_index(),nganh[['Ticker','ICB2','ICB3','ICB4','ICB5']],on='Ticker')
df.set_index(['Ticker','year','quarter'],inplace=True)

In [10]:
# thep = df[df['ICB5']=='Sản xuất, chế biến thép L5']
def add_ratios(x):
    x['jv_income'] = x['jv_inc1'] + x['jv_inc2']
    x['cip'] = x['cip1']
    x['tax_rate'] = 1 - x['net_income'] / (1+x['pretax_inc'])
    x['op'] = x['gp'] - np.abs(x['admin_exp']) - np.abs(x['sel_exp'])
    x['op1'] = x['op'] + x['jv_income']
    x['EBT'] = x['op'] - np.abs(x['int_exp'])
    x['fin_income'] = x['fin_rev'] - (np.abs(x['fin_exp']) - np.abs(x['int_exp']))
    x['other_income'] = x['other_inc']
    # x['bs_gross_fa'] = x['gross_tfa'] + x['gross_ifa'] + x['gross_lease'] + x['gross_ip']
    x['bs_fa'] = x['net_tfa'] + x['net_ifa'] + x['net_ip'] + x['net_lease']
    x['bs_cash'] = (x['cce'] + x['net_st_inv'])
    x['bs_capex'] = -(x.sort_index(level=[0, 1]).groupby(level=0)["bs_fa"].diff(periods=1) +
                      x.sort_index(level=[0, 1]).groupby(level=0)["cip"].diff(periods=1))
    x['bs_ar'] = x['st_ar']
    x['bs_ap'] = x['st_trade_ap']
    x['bs_cust_pre'] = x['st_prepaid_cust'] + x['lt_prepaid_cust'] + x['st_unrev'] + x['lt_unrev']
    x['debt'] = x['st_debt'] + x['lt_debt']
    x['core_e'] = x['op'] * (1 - x['tax_rate'])
    x['op2'] = x['core_e'] + x['jv_income']
    x['netcash'] = (x['bs_cash'] - x['cl'])
    x['ic'] = x['te'] + x['debt']
    x['other_asset'] = x['ta'] - x['bs_cash'] - x['bs_ar'] - x['net_inven'] - x['bs_fa'] - x['cip']-x['lt_inv']
    x['other_lia'] = x['tl'] - x['debt'] - x['bs_ap'] - x['bs_cust_pre']
    x['other_equity'] = x['te'] - x['re'] - x['cap_e'] - x['bs_treasury']
    x['div'] = x['cf_div'] + x['cf_treasury']
    x['cf_delta_debt'] = x['cf_divay'] + x['cf_debt_paid'] + x['cf_lease_paid']
    x['cf_khac'] = (x['cfo'] + x['cfi'] + x['cff']) - (x['cf_capex'] + x['net_income'] + x['cf_dep']
                                                       + x['cf_e_raise'] + x['cf_delta_debt'] + x['div'])
add_ratios(df)
# thep = thep.loc[idx[:,:,5],['rev','gp','sel_exp','admin_exp','jv_inc1','jv_inc2','cf_dep','int_exp','cf_int_paid','st_debt','lt_debt','st_prepaid_cust','st_unrev','lt_prepaid_cust','lt_unrev','ta','te','tl','net_income','bs_fa','bs_cash','debt']]/10**9
# thep.reset_index().to_excel('congtythep.xlsx')

In [11]:
df.loc[idx['TMS',2015:,5],['rev','gp','sel_exp','admin_exp','jv_inc1','jv_inc2','cf_dep','int_exp','cf_int_paid','st_debt','lt_debt','st_prepaid_cust','st_unrev','lt_prepaid_cust','lt_unrev','ta','te','tl','net_income']]/10**9

rev      gp sel_exp admin_exp jv_inc1 jv_inc2  \
Ticker year quarter                                                       
TMS    2015 5.000     488.804 121.547  -0.931   -28.276  85.117   0.000   
       2016 5.000     615.661 154.097  -0.181   -42.425  98.394   0.000   
       2017 5.000   2,131.809 249.327 -13.240   -59.715  99.389   0.000   
       2018 5.000   2,323.712 233.827 -17.006   -68.910 163.927   0.000   
       2019 5.000   2,340.091 272.830 -15.539   -80.137 104.219   0.000   
       2020 5.000   3,418.341 306.873 -12.390   -98.228 170.835   0.000   
       2021 5.000   6,381.266 546.257 -55.163  -159.649 372.116     NaN   
       2022 5.000   3,648.088 549.942 -29.783  -173.874 353.784   0.000   

                     cf_dep int_exp cf_int_paid st_debt lt_debt  \
Ticker year quarter                                               
TMS    2015 5.000    36.007 -15.462      -8.491  58.070 395.281   
       2016 5.000    44.086 -29.492     -44.417 123.836 673.509   
       2017 5.000    66.128 -58.683     -61.291 263.061 517.329   
       2018 5.000    72.019 -62.054     -53.450 194.943 796.924   
       2019 5.000    86.197 -65.984     -70.605 276.926 603.497   
       2020 5.000    95.536 -53.626     -63.248 310.096 533.607   
       2021 5.000   108.302 -54.953     -56.464 716.762 583.265   
       2022 5.000   124.576 -58.089     -59.888 278.622 545.536   

                    st_prepaid_cust st_unrev lt_prepaid_cust lt_unrev  \
Ticker year quarter                                                     
TMS    2015 5.000             0.005    3.075           0.000    0.000   
       2016 5.000             0.036    3.018           0.000    0.000   
       2017 5.000             1.003    2.821           0.000    0.000   
       2018 5.000             2.644    2.480           0.000    0.000   
       2019 5.000             1.627    1.775           0.000    0.000   
       2020 5.000             1.812    0.665           0.000    0.000   
       2021 5.000             3.763    0.536           0.000    0.000   
       2022 5.000             7.051    1.192           0.000    0.000   

                           ta        te        tl net_income  
Ticker year quarter                                           
TMS    2015 5.000   1,420.069   868.259   551.810    155.372  
       2016 5.000   2,006.279 1,078.388   927.890    172.197  
       2017 5.000   2,737.361 1,421.715 1,315.646    211.414  
       2018 5.000   3,166.212 1,625.789 1,540.424    235.012  
       2019 5.000   3,310.259 1,947.165 1,363.094    225.324  
       2020 5.000   3,919.585 2,346.510 1,573.075    321.629  
       2021 5.000   5,734.084 3,489.916 2,244.168    620.900  
       2022 5.000   5,570.603 4,147.586 1,423.018    682.315

In [28]:
df.loc[idx[['GMD','VSC','TMS','SGP','PHP'],2022:,5],['ta','tl','te','debt']]/10**9

,,,ta,tl,te,debt
Ticker,year,quarter,,,,
GMD,2022,5.000,"13,030.653","5,082.897","7,947.756","2,029.579"
VSC,2022,5.000,"4,366.810","1,165.610","3,201.200",908.400
TMS,2022,5.000,"5,570.603","1,423.018","4,147.586",824.158
SGP,2022,5.000,"5,396.366","2,709.199","2,687.167",318.620
PHP,2022,5.000,"6,615.371","1,232.593","5,382.778",577.138


In [ ]:
df[df['ICB5'] == 'Sản xuất, chế biến thép L5'].loc[idx[:,2018:,[1,2,3,4]],['rev','net_income']].groupby(['year','quarter']).apply(lambda x: x.count())
# .loc[idx[['CTD','HBC','FCN','CC1','S99'],2018:,[1,2,3,4]],['net_income']].unstack(0)/10**9
# .groupby(['year','quarter']).apply(lambda x: x['rev'].sum())

# CƠ CẤU CHI PHÍ THEO YẾU TỐ

In [ ]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=42.112.22.56;'
                      'Database=SDv2;'
                      'UID=FiinRate.SDv2.View;'
                      'PWD=FiinR@tings1234')

In [20]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=42.112.22.30,63866;'
                      'Database=FiinRatings;'
                      'UID=FiinRatings.FRA.View;'
                      'PWD=Fiin@1212')

In [21]:
query = """SELECT Distinct
--b.RecordId,
--b.BalanceSheetId,
b.OrganizationId,
a.TaxCode,
a.OrganizationName,
t.Ticker,
P.RealYear as yearreport,
p.RealQuarter as lengthreport,
n.NC160, --chi phi san xuat theo yeu,
n.NC161, --chi phi nguyen lieu vat lieu,
n.NC162, --chi phi nhan cong
n.NC163,--chi phi khau hao tai san co dinh
n.NC164,--chi phi dich vu mua ngoai
n.NC165--chi phi khac bang tien

from [FiinRatings].[dbo].[FGFB_COFS_DR_BalanceSheet] b
join [FiinRatings].[dbo].[FGFB_COIN_DR_ListedInformation] t on b.OrganizationId = t.OrganizationID
left join [FiinRatings].[dbo].[FGFB_COIN_DR_Organization] a on b.OrganizationId = a.OrganizationID
left join [FiinRatings].[dbo].[FGFB_COFS_DR_NoteCompany] n on
b.OrganizationID = n.OrganizationID and 
b.PeriodID = n.PeriodID and 
b.IsPriority = n.IsPriority and 
b.IsHistory = n.IsHistory and
--b.IsPriority = 1 and b.IsHistory = 0 and 
b.ReportTypeId = n.ReportTypeId
INNER JOIN FGFB_SYDI_DR_Period P ON B.PeriodId = P.PeriodId
INNER JOIN FGFB_SYDI_DM_PeriodType Pt ON p.PeriodTypeId = PT.PeriodTypeId
INNER JOIN FGFB_SYDI_DM_FiscalType PF ON p.FiscalTypeId = PF.FiscalTypeId


where 
b.IsPriority = 1 and b.IsHistory = 0
and b.ReportTypeId in (2,3) --2: BCTC hợp nhất kiểm toán, 3: BCTC hợp nhất chưa kiểm toán, 4: BCTC riêng lẻ kiểm toán, 5: BCTC riêng lẻ chưa kiểm toán00
and p.RealQuarter in (1,2,3,4) -- như lengthreport 

--and p.RealYear in (2018,2019,2020,2021,2022) -- đây là year report
and
t.Ticker in (
'SBT'

) 
-- lookup từ mã số thuế theo bảng FGFB__COIN_DR_Organization


"""

In [22]:
df = pd.read_sql_query(query,conn)

In [23]:
note = {'NC160':'tong_chiphi',
        'NC161':'chiphi_nguyenvatlieu',
        'NC162':'chiphi_nhancong',
        'NC163':'chiphi_khauhao',
        'NC164':'chiphi_muangoai',
        'NC165':'chiphi_khac',
       'yearreport':'year',
       'lengthreport':'quarter'}

In [24]:
df = df.rename(columns=note)

In [28]:
df.to_excel('SBT.xlsx')

,OrganizationId,TaxCode,OrganizationName,Ticker,year,quarter,tong_chiphi,chiphi_nguyenvatlieu,chiphi_nhancong,chiphi_khauhao,chiphi_muangoai,chiphi_khac
0,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2007,1,NaN,NaN,NaN,NaN,NaN,NaN
1,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2007,2,NaN,NaN,NaN,NaN,NaN,NaN
2,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2007,3,NaN,NaN,NaN,NaN,NaN,NaN
3,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2007,4,NaN,NaN,NaN,NaN,NaN,NaN
4,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2008,1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
65,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2022,3,NaN,NaN,NaN,NaN,NaN,NaN
66,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2022,3,0.000,0.000,0.000,0.000,0.000,0.000
67,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2022,4,NaN,NaN,NaN,NaN,NaN,NaN
68,3706,3900244389,Công ty Cổ phần Thành Thành Công - Biên Hòa,SBT,2022,4,0.000,0.000,0.000,0.000,0.000,0.000


In [ ]:
df = df.set_index(['Ticker','year','quarter'])